# Physics of Complex Systems final project

Simulate a crossover between directed percolation and compact directed percolation. Find critical point, study critical exponents at and near critical point.

In [ ]:
import time
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def initial_condition(row, initial_energy=100):
    """ 
    Sets the initial energy of the site in the center
    """
    row[int(len(row)/2)] += initial_energy
    return row


def activation(row_t, row_t1, Eb, dissipation):
    """
    Calculate the energy transferred to the grains of the next row t+1
    
    """
    active_sites = list(np.where(row_t > Eb))
    # For each active site, give energy to the 3 below. 
    flag = False # true if we reach lateral boundaries, used to compute correlation leghts even if we reach boundaries
    for site in active_sites[0]:
        #print(f'site: {site}')
        if site == 0 or site == len(row_t)-1: 
            print('Reached boundary for site: ', site)
            row_t1 = np.zeros(len(row_t))
            flag = True # still compute correlation lengths 
            return row_t1, flag # boundary condition to avoid break

        # generate 3 random numbers, represent fraction of energy trasnmitted to each grain
        r_1 = np.random.rand()
        r0 = np.random.rand()
        r1 = np.random.rand()
        norm = r_1+r0+r1

        delta_E = row_t[site]*(1-dissipation) # fraction of energy not dissipated and transmitted to neighbours

        z_1 = r_1/norm
        z0 = r0/norm
        z1 = r1/norm
        # assign random energies to neighbours
        row_t1[site-1] += z_1*delta_E
        row_t1[site] += z0*delta_E
        row_t1[site+1] += z1*delta_E

    active_sites_t1 = list(np.where(row_t1 > Eb))
    for site in active_sites_t1[0]: 
        row_t1[site] +=1 # give potential energy   
    return row_t1, flag

def percolation(N, t, Eb, f, plot = False, tangent = False):
    """
    Simulation of one percolation event. The plot argument allows for plotting, tangent computes the parallel and orthogonal correlation lenghts
    """
    N_t = np.zeros(t)
    # initialize matrix
    grid = np.zeros((t,N))
    while np.all(grid[0]==0):
        grid[0] = initial_condition(grid[0], initial_energy=100)
    N_t[0] = 1

    for step in range(t-1):
        #for each time step, activate on next row
        row_t, flag = activation(grid[step], grid[step+1], Eb, f) #(row(t), row(t+1), Eb, f)
        grid[step+1] = row_t 
        
        N_t[step+1] = len(np.where(grid[step+1] > Eb)[0]) # N(t)
 
        # absorbing phase 
        if np.all(grid[step+1]<Eb):
            print('Absorbing phase')
            ext_time=step+1
            Eb_c = Eb
            # correlation lenghts in absorbing phase, keep them only if we reached boundaries (flag = T)
            if tangent:
                xi_parallel = t
                right = np.max(np.where(grid > Eb)[1])
                left = np.min(np.where(grid > Eb)[1])
                xi_ortho = .5*(right - left)
                if flag: return xi_parallel, xi_ortho, 0, 0 # late absorbing phase, compute correlation lengths 
                else: return 0, 0, 0, 0 # early absorbing phase, discard
            return grid, Eb_c, N_t, ext_time
    
    # correlation lenghts
    if tangent : 
        xi_parallel = t  
        right = np.max(np.where(grid > Eb)[1])
        left = np.min(np.where(grid > Eb)[1])
        xi_ortho = .5*(right - left) 
        return xi_parallel, xi_ortho, 0, 0

    if plot:
        mask = [el > 0 for el in grid]
        grid_plot = grid
        grid_plot[mask] = 1
        plt.matshow(grid_plot[:step], cmap = 'Greys', aspect = 'auto') # to have black and white grid of 0 and 1 for activation
        plt.title(f'Eb = {Eb:.3f}') 
    return grid, 0, N_t, step 


## Find phase diagram of (f,Eb) plane

In [ ]:
# Vary the dissipation to find the corresponding critical energy value

N = 1000
t = 5000
Eb_range = np.linspace(0.3855,.3865, 5)
Eb_c_rep = []
#f_range = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9,1]
f = 0.5
ntry = 5 # simulate 5 times for each Eb, to reach true absorbing phase
n_reps = 10
for rep in range(n_reps):
    for Eb in Eb_range:
        count = 0

        for n in range(ntry):
            _, Eb_c1, _, _ = percolation(N, t, Eb, f, plot=False)
            if Eb_c1 != 0: # absorbing phase
                #print(f'Absorbed {n+1} times for {Eb_c1:.6f}')
                count +=1

        if count == ntry: # true absorbing phase
            Eb_c = Eb_c1
            print(f'Critical value: {Eb_c:.6f}')
            Eb_c_rep.append(Eb_c)
            break   

print(f'Critical value: {np.mean(Eb_c_rep):.6f}')

In [ ]:
# Phase diagram plot

f_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9,1]
Ebc_list = [3.27,2.12,1.548,1.21,0.96,0.794,0.656,0.558,0.464,0.386,0.271,0.179,0.109,0.052,0]

plt.scatter(f_list, Ebc_list, color='firebrick', marker='d')
plt.plot(f_list, Ebc_list, label='Simulation', color='firebrick')

f = np.linspace(0,1)
y = 2*(1-f)/(1+2*f)

plt.plot(f,y, c='black', label='Mean Field', linestyle='-.', linewidth=1)
plt.xlabel('f')
plt.ylabel('Eb')
plt.legend()

To create percolation plots

In [ ]:
N = 3000
t = 3000
f = 0.5
Ebc = 0.385997

img = 1
start_time = time.time()

flag = 1 # Ensure convergence
count = 0
while flag != 0 :
    grid, flag ,_, _ = percolation(N, t, Ebc, f, False, False)
    count +=1
    #if count % 5 == 0: print(f'{count} iterations \n')
    print(f'{count} iterations \n')
print(f'Total time: {time.time() - start_time:.3f}')
mask = [el > 0 for el in grid]
grid_plot = grid
grid_plot[mask] = 1
fig = plt.figure(figsize=(5,8))
plt.matshow(grid_plot[:,1450:1550], fignum=0, cmap = 'Greys', aspect = 'auto') # focus in middle section of grid
plt.savefig(f'f05_{t}_{img}.png')
plt.close()


## Fix f=0.5, find opening of cone as a function of Eb

In [ ]:
N = 3000
t = 3000
f = 0.5
Ebc = 0.3
start_time = time.time()

flag = 1 # Ensure convergence, percolation returns flag = 0 for absorbing phase
count = 0
while flag != 0 :
    grid, flag ,_, _ = percolation(N, t, Ebc, f, False, False)
    count +=1
    #if count % 5 == 0: print(f'{count} iterations \n')
    print(f'{count} iterations \n')
print(f'Total time: {time.time() - start_time:.3f}')

#  plot the correlation lenghts
right = np.max(np.where(grid > Ebc)[1])
left = np.min(np.where(grid > Ebc)[1])

mask = [el > 0 for el in grid]
grid_plot = grid
grid_plot[mask] = 1
plt.matshow(grid_plot, cmap = 'Greys', aspect = 'auto')
plt.axvline(left, label = 'left')
plt.axvline(right, label = 'right')
plt.axvline(int(len(grid[0])/2), label = 'middle', color = 'r')
plt.title(f'Eb = {Ebc}')
plt.legend()
plt.savefig(f'tangent_{Ebc}.png')

### Study finite size effects increasing the dimension of the grid

In [ ]:
# parameters

N = 3000
t = 3000
f = 0.5
Ebc = 0.386
sims = [500,200,150,100,50,50,50,25]
delta_e = [0.01,0.015,0.02,0.025,0.03,0.05,0.1,0.11]
en_count = 0
xi_parallel_avg = []
xi_ortho_avg = []
start_time = time.time()

for energy in delta_e:
    xi_parallel = []
    xi_ortho = []
    for sim in range(sims[en_count]):
        if sim %10 ==0: print(f'Simulation: {sim}')
        xi_p_temp, xi_o_temp ,_, _ = percolation(N, t, Ebc-energy, f, False, True) # compute correlation lemghts
        
        xi_parallel.append(xi_p_temp)
        xi_ortho.append(xi_o_temp)
    # compute average    
    xi_parallel_avg.append(np.mean(xi_parallel)) 
    xi_ortho_avg.append(np.mean(xi_ortho))   
    print(f'Time: {time.time() - start_time:.3f} \n')    
    en_count += 1

print(f'Total time: {time.time() - start_time:.3f}')

tan_theta = [i/j for i,j in zip(xi_ortho_avg, xi_parallel_avg)] # compute the tangent of the opening angle

points = np.linspace(min(delta_e),max(delta_e)) # to plot interpolation line

plt.scatter(delta_e, tan_theta, color = 'white', edgecolors='black')
plt.plot(points, 3.2*points, linestyle = '-.', color = 'black')
plt.xlabel('Eb - Eb_c')
plt.ylabel('tan($\\theta$)')
plt.title(f'$T_max$ = {t}')
plt.loglog()
plt.savefig(f'tan_{t}_sims.png')

In [ ]:
# parameters

N = 10000
t = 15000
f = 0.5
Ebc = 0.386
sims = [300,150,100,75,50,20,15]
delta_e = [0.01,0.015,0.02,0.025,0.03,0.05,0.1]
en_count = 0
xi_parallel_avg = []
xi_ortho_avg = []
start_time = time.time()

for energy in delta_e:
    xi_parallel = []
    xi_ortho = []
    for sim in range(sims[en_count]):
        if sim %10 ==0: print(f'Simulation: {sim}')
        xi_p_temp, xi_o_temp ,_, _ = percolation(N, t, Ebc-energy, f, False, True) # compute correlation lemghts
        
        xi_parallel.append(xi_p_temp)
        xi_ortho.append(xi_o_temp)
    # compute average    
    xi_parallel_avg.append(np.mean(xi_parallel)) 
    xi_ortho_avg.append(np.mean(xi_ortho))   
    print(f'Time: {time.time() - start_time:.3f} \n')    
    en_count += 1

print(f'Total time: {time.time() - start_time:.3f}')

tan_theta = [i/j for i,j in zip(xi_ortho_avg, xi_parallel_avg)] # compute the tangent of the opening angle

points = np.linspace(min(delta_e),max(delta_e)) # to plot interpolation line

plt.scatter(delta_e, tan_theta, color = 'white', edgecolors='black')
plt.plot(points, 3.2*points, linestyle = '-.', color = 'black')
plt.xlabel('Eb - Eb_c')
plt.ylabel('tan($\\theta$)')
plt.title(f'$T_max$ = {t}')
plt.loglog()
plt.savefig(f'tan_{t}_sims.png')

## Retrieve N(t) and P(t) to find critical exponents

Check finite size effects on N(t) and P(t) increasing grid size

In [ ]:
# parameters
N = 3000
t = 3000
f = 0.5
Eb = 0.38599
n_sim = 2500

# critical exponents
eta_cdp = 0
eta_dp = 0.3137
delta_cdp = 0.5
delta_dp = 0.1595

N_t = np.zeros((t,n_sim))
surv_prob = np.ones((t,n_sim))

# simulate the percolation, store the number of active sites and the survival probability
for sim in range(n_sim):
    _, _, N_t[:,sim], ext_time = percolation(N, t, Eb, f, False)
    surv_prob[ext_time:,sim] = 0
    if sim %50 ==0: print(f'Simulation: {sim}')

t_range = np.arange(0,t)

# compute average
N_t_avg = np.mean(N_t, axis=1)
surv_prob_avg = np.mean(surv_prob, axis=1)

# plot the results, adjust intervals and slopes 
plt.plot(t_range, N_t_avg, label='N(t)', color = 'b')
plt.plot(t_range,surv_prob_avg, label='P(t)', color = 'red')
plt.plot(t_range[1000:], .9*t_range[1000:]**eta_dp, color='green', label='DP')
plt.plot(t_range[10:80], 6.3*t_range[10:80]**eta_cdp, color='orange', label='CDP')
plt.plot(t_range[2500:], 1*t_range[2500:]**(-1*delta_dp), color='green')
plt.plot(t_range[40:120], 6.5*t_range[40:120]**(-1*delta_cdp), color='orange')
plt.title(f'$T max$ = {t}, {n_sim} simulations')
plt.xlabel('t')
plt.legend()
plt.loglog()
plt.savefig(f'N_P_t{t}_sim{n_sim}.png')

In [ ]:
# parameters
N = 7000
t = 15000
f = 0.5
Eb = 0.38599
n_sim = 2500

# critical exponents
eta_cdp = 0
eta_dp = 0.3137
delta_cdp = 0.5
delta_dp = 0.1595

N_t = np.zeros((t,n_sim))
surv_prob = np.ones((t,n_sim))

# simulate the percolation, store the number of active sites and the survival probability
for sim in range(n_sim):
    _, _, N_t[:,sim], ext_time = percolation(N, t, Eb, f, False)
    surv_prob[ext_time:,sim] = 0
    if sim %50 ==0: print(f'Simulation: {sim}')

t_range = np.arange(0,t)

# compute average
N_t_avg = np.mean(N_t, axis=1)
surv_prob_avg = np.mean(surv_prob, axis=1)

#plot results
plt.plot(t_range, N_t_avg, label='N(t)', color = 'b')
plt.plot(t_range,surv_prob_avg, label='P(t)', color = 'red')
plt.plot(t_range[1000:], .9*t_range[1000:]**eta_dp, color='green', label='DP')
plt.plot(t_range[10:80], 6.3*t_range[10:80]**eta_cdp, color='orange', label='CDP')
plt.plot(t_range[3000:], 1*t_range[3000:]**(-1*delta_dp), color='green')
plt.plot(t_range[40:120], 6.5*t_range[40:120]**(-1*delta_cdp), color='orange')
plt.title(f'$T max$ = {t}, {n_sim} simulations')
plt.xlabel('t')
plt.legend()
plt.loglog()
plt.savefig(f'N_P_t{t}_sim{n_sim}.png')

In [ ]:
# parameters
N = 10000
t = 35000
f = 0.5
Eb = 0.386
n_sim = 1500

# critical exponents
eta_cdp = 0
eta_dp = 0.3137
delta_cdp = 0.5
delta_dp = 0.1595

N_t = np.zeros((t,n_sim))
surv_prob = np.ones((t,n_sim))

# simulate the percolation, store the number of active sites and the survival probability
for sim in range(n_sim):
    _, _, N_t[:,sim], ext_time = percolation(N, t, Eb, f, False)
    surv_prob[ext_time:,sim] = 0
    if sim %50 ==0: print(f'Simulation: {sim}')

t_range = np.arange(0,t)

# compute average
N_t_avg = np.mean(N_t, axis=1)
surv_prob_avg = np.mean(surv_prob, axis=1)

#plot results
plt.plot(t_range, N_t_avg, label='N(t)', color = 'b')
plt.plot(t_range,surv_prob_avg, label='P(t)', color = 'red')
plt.plot(t_range[1000:], 1.1*t_range[1000:]**eta_dp, color='green', label='DP')
plt.plot(t_range[5:100], 7.5*t_range[5:100]**eta_cdp, color='orange', label='CDP')
plt.plot(t_range[3000:], 0.8*t_range[3000:]**(-1*delta_dp), color='green')
plt.plot(t_range[30:100], 6*t_range[30:100]**(-1*delta_cdp), color='orange')
plt.title(f'$T max$ = {t}, {n_sim} simulations')
plt.legend()
plt.loglog()
plt.savefig(f'N_P_t{t}_sim{n_sim}.png')

In [ ]:
# parameters
N = 20000
t = 200000
f = 0.5
Eb = 0.38599
n_sim = 1500

# critical exponents
eta_cdp = 0
eta_dp = 0.3137
delta_cdp = 0.5
delta_dp = 0.1595

N_t = np.zeros((t,n_sim))
surv_prob = np.ones((t,n_sim))

# simulate the percolation, store the number of active sites and the survival probability
for sim in range(n_sim):
    _, _, N_t[:,sim], ext_time = percolation(N, t, Eb, f, False)
    surv_prob[ext_time:,sim] = 0
    if sim %50 ==0: print(f'Simulation: {sim}')

t_range = np.arange(0,t)

# compute average
N_t_avg = np.mean(N_t, axis=1)
surv_prob_avg = np.mean(surv_prob, axis=1)

#plot results
plt.plot(t_range, N_t_avg, label='N(t)', color = 'b')
plt.plot(t_range,surv_prob_avg, label='P(t)', color = 'red')
plt.plot(t_range[1000:], .9*t_range[1000:]**eta_dp, color='green', label='DP')
plt.plot(t_range[10:80], 6.3*t_range[10:80]**eta_cdp, color='orange', label='CDP')
plt.plot(t_range[3000:], 1*t_range[3000:]**(-1*delta_dp), color='green')
plt.plot(t_range[40:120], 6.5*t_range[40:120]**(-1*delta_cdp), color='orange')
plt.title(f'$T max$ = {t}, {n_sim} simulations')
plt.xlabel('t')
plt.legend()
plt.loglog()
plt.savefig(f'N_P_t{t}_sim{n_sim}.png')